In [10]:
%load_ext autoreload
%autoreload 2
import numpy as np
from final_algo import (GET_ALGOS,
                        #new_algo, 
                        #anandkumar_algo, nbsel, glasso_vanilla, stability_wrapper,
                        SH_stability_wrapper, 
                        long_lambdas, short_lambdas, super_short_lambdas, get_SH_lambdas)
from paper_sims_util import GET_GRAPHS, MCC
import time
from collections import namedtuple

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
GraphParams = namedtuple('GraphParams', 'N eta p d ratios')
AlgoParams = namedtuple('AlgoParams', 'stability_samples M pi')

In [25]:
graph_params_dict = {
    'chain': GraphParams(p=50, N=[20, 25, 30, 35, 40], eta=1, ratios=None, d=None), #p, N, eta
    'star': GraphParams(p=50, d=[10, 15, 20, 25, 30], N=50, eta=1, ratios=None), #p, d, N, eta
    'random': GraphParams(p=50, d=0.01, ratios=[r/500. for r in [300, 375, 500, 750, 1000]], eta=1, N=None), #p, d, ratio over 500, eta
    'grid_3D': GraphParams(p=4, ratios=[r/524. for r in [200, 250, 300, 400, 500]], eta=2, N=None, d=None), #p, ratio over 524, eta
    'grid': GraphParams(p=7, ratios=[r/529. for r in [75, 100, 150, 200, 250]], eta=2, N=None, d=None) #p, ratio over 529, eta
}

algo_params = AlgoParams(stability_samples=50, M=7./9., pi=0.9)
run_id = 'test_run'

In [ ]:
for graph_name in ['chain', 'star', 'random', 'grid_3D', 'grid']:
    graph_params = graph_params_dict[graph_name]
    wrapper[graph_name](graph_params, algo_params, run_id)

In [ ]:
def get_results(graph_params, algo_params, omega, N):
    algos = GET_ALGOS(algo_params.stability_samples)
    algo_lambdas = get_algo_lambdas(algo_params.M, graph_params.eta, N, graph_params.p)
    sigma = np.linalg.inv(omega)
    X = np.random.multivariate_normal(mean = np.zeros(omega.shape[0]), 
                                      cov = np.linalg.inv(omega), 
                                      size = N)
    
    ALL_RESULTS = {}
    for algo_name, algo in algos.items():
        print("Currently on {}".format(algo_name))
        lambdas = algo_lambdas[algo_name]
        if algo_name == 'our':
            results = algo(X, algo_params.M)
        else:
            results = algo(X, lambdas, algo_params.pi)
        ALL_RESULTS[algo_name] = results
        omega_hat = results[0]
        res_mcc = MCC(omega_hat, omega)
        print("Algorithm {} got {} MCC".format(algo_name, res_mcc))
    return ALL_RESULTS

In [35]:
def star_wrapper(graph_params, algo_params, run_id):
    p = graph_params.p
    for d in graph_params.d:
        omega = star(d, p)
        results = get_results(graph_params, algo_params, omega, graph_params.N)
        fname = "{}_{}_{}_results.pkl".format(run_id, 'star', d)
        with open(fname, 'wb') as f:
            pickle.dump(results, f)

In [30]:
def grid_3D_wrapper(graph_params, algo_params, run_id):
    p = graph_params.p
    dim = p**3
    Ns = [int(r*dim) for r in graph_params.ratios]
    for r, N in zip(graph_params.ratios, Ns):
        omega = grid_3D(p)
        results = get_results(graph_params, algo_params, omega, N)
        fname = "{}_{}_{}_results.pkl".format(run_id, 'grid_3D', r)
        with open(fname, 'wb') as f:
            pickle.dump(results, f)

In [32]:
def grid_wrapper(graph_params, algo_params, run_id):
    p = graph_params.p
    dim = p**2
    Ns = [int(r*dim) for r in graph_params.ratios]
    for r, N in zip(graph_params.ratios, Ns):
        omega = grid(p)
        results = get_results(graph_params, algo_params, omega, N)
        fname = "{}_{}_{}_results.pkl".format(run_id, 'grid', r)
        with open(fname, 'wb') as f:
            pickle.dump(results, f)

In [33]:
def chain_wrapper(graph_params, algo_params, run_id):
    p = graph_params.p
    for N in graph_params.N:
        omega = chain(p)
        results = get_results(graph_params, algo_params, omega, N)
        fname = "{}_{}_{}_results.pkl".format(run_id, 'chain', N)
        with open(fname, 'wb') as f:
            pickle.dump(results, f)

In [34]:
def random_wrapper(graph_params, algo_params, run_id):
    p = graph_params.p
    d = graph_params.d
    dim = p
    Ns = [int(r*dim) for r in graph_params.ratios]
    for r, N in zip(graph_params.ratios, Ns):
        omega = random_graph(p,d)
        results = get_results(graph_params, algo_params, omega, N)
        fname = "{}_{}_{}_results.pkl".format(run_id, 'random', r)
        with open(fname, 'wb') as f:
            pickle.dump(results, f)

In [13]:
graph_type = 'random'
d = 0.01
p = 20
#omega = GRAPHS[graph_type](p, d)
omega = random_graph(p,d)
sigma = np.linalg.inv(omega)

#stability selection
NUM_SUBSAMPLES = 50
pi = 0.9
#our algorithm
M = 7./9.

ratios = [300, 375, 500, 750, 1000]
list_of_Ns = [int(r/500 * p) for r in ratios]

print(list_of_Ns)

start = time.time()
for N in list_of_Ns:
    X = np.random.multivariate_normal(mean = np.zeros(p), cov = np.linalg.inv(omega), size = N)
    ALGOS = GET_ALGOS(NUM_SUBSAMPLES)
    algo_lambdas = {
        'our': 7./9.,
        'SH': get_SH_lambdas(),
        'glasso': [0.1, 0.3, 0.5, 1.0],#long_lambdas(N, p),
        'nbsel': [0.1, 0.3, 0.5, 1.0],
        'anand': super_short_lambdas(N, p)
    }
    results = get_results(ALGOS, algo_lambdas)
    #pickle.save(results)
end = time.time()
print(end-start)

[12, 15, 20, 30, 40]
Currently on our
Running new algorithm
N=12, M=6
Working on l = 0
Working on l = 1
Working on l = 2
Algorithm our got 0.4424558687227414 MCC
Currently on SH
Algorithm SH got -0.007502485610105728 MCC
Currently on glasso
Working on 0.1
graphical_lasso FloatingPointError: 0.1
Working on 0.3
graphical_lasso ConvergenceWarning: 0.3
Working on 0.5
Working on 1.0
Algorithm glasso got 0 MCC
Currently on nbsel
Working on 0.1
Working on 0.3
Working on 0.5
Working on 1.0
Algorithm nbsel got -0.005291005291005291 MCC
Currently on anand
Working on 0.049964422955689106
Working on 0.49964422955689103
Working on 3.9971538364551282
Algorithm anand got 0 MCC
Currently on our
Running new algorithm
N=15, M=8
Working on l = 0
Working on l = 1
Working on l = 2
Algorithm our got -0.007502485610105728 MCC
Currently on SH
Algorithm SH got 0.5742873814325569 MCC
Currently on glasso
Working on 0.1
graphical_lasso ConvergenceWarning: 0.1
Working on 0.3
graphical_lasso ConvergenceWarning: 0.3

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Working on 0.3
Working on 0.5
Working on 1.0
Algorithm nbsel got 0 MCC
Currently on anand
Working on 0.044689538474188724
Working on 0.4468953847418872
Working on 3.575163077935098
Algorithm anand got 1.0 MCC
Currently on our
Running new algorithm
N=20, M=10
Working on l = 0
Working on l = 1
Working on l = 2
Algorithm our got -0.007502485610105728 MCC
Currently on SH
Algorithm SH got 0.5742873814325569 MCC
Currently on glasso
Working on 0.1
graphical_lasso OverflowError: 0.1
Working on 0.3
graphical_lasso ConvergenceWarning: 0.3
Working on 0.5
Working on 1.0
Algorithm glasso got 0 MCC
Currently on nbsel
Working on 0.1


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Working on 0.3
Working on 0.5
Working on 1.0
Algorithm nbsel got -0.005291005291005291 MCC
Currently on anand
Working on 0.0387022756020495
Working on 0.38702275602049496
Working on 3.0961820481639597
Algorithm anand got 0 MCC
Currently on our
Running new algorithm
N=30, M=14
Working on l = 0
Working on l = 1
Working on l = 2
Algorithm our got 0.4424558687227414 MCC
Currently on SH
Algorithm SH got 0.4424558687227414 MCC
Currently on glasso
Working on 0.1
graphical_lasso ConvergenceWarning: 0.1
Working on 0.3
Working on 0.5
Working on 1.0
Algorithm glasso got 1.0 MCC
Currently on nbsel
Working on 0.1
Working on 0.3
Working on 0.5
Working on 1.0
Algorithm nbsel got 0.7052336473499384 MCC
Currently on anand
Working on 0.031600275703195964
Working on 0.3160027570319596
Working on 2.528022056255677
Algorithm anand got 0.7052336473499384 MCC
Currently on our
Running new algorithm
N=40, M=17
Working on l = 0
Working on l = 1
Algorithm our got 0.5742873814325569 MCC
Currently on SH
Algorithm 

In [4]:
def get_results(algos, algo_lambdas):
    results = {}
    for algo_name, algo in algos.items():
        print("Currently on {}".format(algo_name))
        lambdas = algo_lambdas[algo_name]
        if algo_name == 'our':
            algo_res = algo(X, M)
        else:
            algo_res = algo(X, lambdas, pi)
        results[algo_name] = algo_res
        res_mcc = MCC(algo_res, omega)
        print("Algorithm {} got {} MCC".format(algo_name, res_mcc))
    return results